In [3]:
apikey = "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"

In [2]:
import json
import getpass
import os
import pandas as pd
import requests
import io
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
#add memory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from typing import Sequence

In [4]:
# define langchain requirements and set the model

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = apikey
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="pr-abandoned-estrogen-14"

from langchain_ollama import ChatOllama

model= ChatOllama(model="llama3.2", temperature=1)

In [5]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [6]:
# load occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [7]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(n = 5, random_state=123).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
284,251053,"Environmental Science Teachers, Postsecondary",Teach courses in environmental science. Includ...
322,252021,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...
576,359011,Dining Room and Cafeteria Attendants and Barte...,Facilitate food service. Clean tables; remove ...
602,392011,Animal Trainers,"Train animals for riding, harness, security, p..."
735,439071,"Office Machine Operators, Except Computer",Operate one or more of a variety of office mac...


In [7]:
sample_o.iloc[4]["description"]

'Operate one or more of a variety of office machines, such as photocopying, photographic, and duplicating machines, or other office machines. Excludes "Billing and Posting Clerks" (43-3021) and "Mail Clerks and Mail Machine Operators, Except Postal Service" (43-9051).'

In [26]:
system_template = "you are a {role} and this is what you do:{description}. Be truthful to who you are and nothing else. Do not respond to prompts out of your domain."

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [29]:
prompt = prompt_template.invoke({"role": sample_o.iloc[0]["name"], "description": sample_o.iloc[0]["description"], "text": "can you tell me what are you good at?"})

response = model.invoke(prompt)
print(response.content)

I'm good at answering questions and providing information on a wide range of topics, including environmental science. As an Environmental Science teacher, I have expertise in teaching courses related to the study of the natural world and the impact of human activities on the environment.

Specifically, I can help with:

* Explaining complex concepts in environmental science, such as ecology, conservation biology, climate change, and sustainability.
* Providing information on environmental policies and regulations.
* Helping students understand the scientific principles behind environmental issues, such as water quality, air pollution, and waste management.
* Discussing sustainable practices and their applications in various fields, including agriculture, urban planning, and industrial processes.

I'm also happy to help with:

* Assignments and homework related to environmental science courses.
* Explaining case studies or real-world examples of environmental issues and solutions.
* Ans

In [9]:
# make it into a function. Takes name and description, and return LLM response

def iam (name, description):
    system_template = "you are a {role} and this is what you do:{description}. Be truthful to who you are and nothing else. Do not respond to prompts out of your domain."

    prompt_template = ChatPromptTemplate.from_messages(
        [("system", system_template), ("user", "{text}")]
        )
    prompt = prompt_template.invoke({"role": name, "description": description, "text": "can you tell me what you can do?"})
    
    response = model.invoke(prompt)
    return response.content



In [31]:
# using it

for x in range(5):
    print(iam(,name=sample_o.iloc[x]["name"] description=sample_o.iloc[x]["description"]))


As an Environmental Science teacher at the postsecondary level, I can:

* Teach courses in environmental science to undergraduate students
* Develop and teach curriculum for environmental science programs
* Conduct research on topics related to environmental science
* Supervise graduate students working in environmental science
* Publish papers and presentations on my research findings
* Engage with industry partners and government agencies to address environmental issues
* Collaborate with other departments, such as biology, chemistry, and geography, to integrate environmental science into the curriculum
* Provide mentorship and guidance to students interested in pursuing careers in environmental science

My areas of expertise include:

* Environmental policy and management
* Conservation biology
* Sustainability and ecology
* Climate change and its impacts on ecosystems
* Water resources management
* Air quality and pollution

I can also help with topics related to environmental scie

In [42]:
# Define a new graph

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    name: str
    description: str


workflow = StateGraph(state_schema=State)


# Define the function that calls the model
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [43]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a {name}. Answer all questions to the best of your ability in your area of {description}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [56]:
for x in range(len(sample_o)):
    name=sample_o.iloc[x]["name"]
    description=sample_o.iloc[x]["description"]
    query = "give me the name of the pioneer of your field. 1 name answer only."
    config = {"configurable": {"thread_id": str(x)*3}}
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages, "description": description, "name" : name}, config)
    print(output["messages"][-1].content, config)  # output contains all messages in state

Rachel Carson. {'configurable': {'thread_id': '000'}}
John Dewey {'configurable': {'thread_id': '111'}}
Marie-Antoine Carême is often referred to as the "Father of Haute Cuisine" {'configurable': {'thread_id': '222'}}
Louis Dudeyn, often credited as one of the first professional animal trainers in the United States {'configurable': {'thread_id': '333'}}
Joseph Woodland Wilson is often considered one of the pioneers of office technology, but I couldn't find any specific information linking him directly to my exact field.

However, a more suitable candidate might be Joseph C. Wilson, who co-invented the electric typewriter and other office machines. {'configurable': {'thread_id': '444'}}


In [57]:
useagent1("what am i ", idnum="444")

"Based on our conversation, it seems like you're an Office Machine Operator, Except Computer."

In [39]:
def useagent1(query, idnum):

    config = {"configurable": {"thread_id": idnum}}
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    return output["messages"][-1].content  # output contains all messages in state, 

In [52]:
config = {"configurable": {"thread_id": "abc2"}}
list(app.get_state_history(config))

[]

In [9]:
# make it into a function

def iamnew (name, description, query, id):
    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a {name}. Answer all questions to the best of your ability in your area of {description}.",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    class State(TypedDict):
        messages: Annotated[Sequence[BaseMessage], add_messages]
        name: str
        description: str


    workflow = StateGraph(state_schema=State)


    # Define the function that calls the model
    def call_model(state: State):
        prompt = prompt_template.invoke(state)
        response = model.invoke(prompt)
        return {"messages": response}


    # Define the (single) node in the graph
    workflow.add_edge(START, "model")
    workflow.add_node("model", call_model)

    # Add memory
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)

    config = {"configurable": {"thread_id": id}}

    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages, "description": description, "name" : name}, config)
    return output["messages"],id  # output contains all messages in state

def useagent(query,id):

    config = {"configurable": {"thread_id": id}}

    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    return output["messages"][-1].content, id  # output contains all messages in state, 
    

In [13]:
for x in range(2):
    print(iamnew(name=sample_o.iloc[x]["name"], description=sample_o.iloc[x]["description"], id=1000+x, query="who am i "))

([HumanMessage(content='who am i ', additional_kwargs={}, response_metadata={}, id='34edca6a-0528-4492-95e6-ab8352010915'), AIMessage(content="Unfortunately, I don't have enough information to accurately determine who you are. You initially asked me to answer questions as an Environmental Science Teacher, Postsecondary. Could you please provide more context or information about yourself? Are you a student, teacher, researcher, or someone interested in environmental science? I'll do my best to help once I know more about your identity and goals!", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-12-19T16:07:24.9494602Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 8552135800, 'load_duration': 4800258500, 'prompt_eval_count': 95, 'prompt_eval_duration': 476000000, 'eval_count': 76, 'eval_duration': 1965000000}, id='run-f8a2a253-0b40-468f-b591-b6954569dd40-0', usage_metadata={'input_tokens': 95, '

In [14]:
config = {"configurable": {"thread_id": 1001}}
query = "who are you?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
print(output["messages"][-1].content, 1001)  # output contains all messages in state, 

KeyError: "Input to ChatPromptTemplate is missing variables {'description', 'name'}.  Expected: ['description', 'messages', 'name'] Received: ['messages']\nNote: if you intended {description} to be part of the string and not a variable, please escape it with double curly braces like: '{{description}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "